### Create True Graph

dataset - http://www.geniaproject.org/genia-corpus/relation-corpus

In [1]:
# !pip install networkx --user
# !pip install pandarallel --user
# !pip install swifter --user

In [2]:
import os
from xml.dom import minidom
import pandas as pd
# from pandarallel import pandarallel as pl
import swifter
from compare_util import iterate_greedy, build_graph, compute_pagerank, iterate_pagerank, iterate_random
import networkx as nx
import operator
from random import *
import numpy as np

pd.set_option('display.max_colwidth', -1)
# pl.initialize()
# pl.initialize(progress_bar=True)

/Users/grace/.local/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
# file_locs = ['/home/grace/Documents/DAEB/jupyter/dataset/BioNLP-ST_2011_Entity_Relations_training_data']
file_locs = ['/Users/grace/workspace/bio_dataset/GENIA_relation_annotation_training_data/BioNLP-ST_2011_Entity_Relations_training_data',
            '/Users/grace/workspace/bio_dataset/GENIA_relation_annotation_test_data/BioNLP-ST_2011_Entity_Relations_test_data/']
total_proteins = []
total_entities = []
sents = []
for file_loc in file_locs:
    medlines = os.listdir(file_loc)
    print(len(medlines))
#     medlines = list(filter(lambda x: x.endswith('.txt'), medlines))
    print(len(medlines))

    for med in medlines:
        f = open(file_loc+'/'+med, 'r')
        if med.endswith('.txt'):
            lines = ' '.join(f.readlines()).replace('\n', ' ')
            sents.append({'id':med.replace('.txt', ''),
                          'sent':lines.lower()})
            
        elif med.endswith('.a1'):
            entities = {}
            for line in f.readlines():
                items = line.split('\t')
#                 print(items)
#                 entity_type = items[1]
#                 entity_value = items[4]
                entities[items[2].strip().lower()] = items[1].split(' ')[0].lower()
    
            total_proteins.append({'id': med.replace('.a1', ''),
                             'protein_val': ' *,* '.join(entities.keys()),
                             'protein_type': ' *,* '.join(entities.values())})
            
        elif med.endswith('.rel'):
            entities = {}
            for line in f.readlines():
                if line.startswith('T'):
                    items = line.split('\t')
#                     print(items)
    #                 entity_type = items[1]
    #                 entity_value = items[4]
                    entities[items[2].strip().lower()] = items[1].split(' ')[0].lower()
    
            total_entities.append({'id': med.replace('.rel', ''),
                             'entity_val': ' *,* '.join(entities.keys()),
                             'entity_type': ' *,* '.join(entities.values())})
        f.close()

2402
2402
522
522


In [4]:
df_sent = pd.DataFrame(sents)
print(df_sent.shape)
df_sent.head()
# df_sent.sent.isnull()

(1060, 2)


,id,sent
0,PMID-8709209,"various modes of basic helix-loop-helix protein-mediated regulation of murine leukemia virus transcription in lymphoid cell lines. the transcriptionally regulatory regions of the lymphomagenic akv and sl3-3 murine leukemia retroviruses (mlvs) contain two types of e-box consensus motifs, cagatg. one type, ea/s, is located in the upstream promoter region, and the other, e(gre), is located in a tandem repeat with enhancer properties. we have examined the requirements of the individual e-boxes in mlv transcriptional regulation. in lymphoid cell lines only, the e(gre)-binding protein complexes included alf1 or heb and e2a basic helix-loop-helix proteins. ectopic alf1 and e2a proteins required intact e(gre) motifs for mediating transcriptional activation. alf1 transactivated transcription of akv mlv through the two e(gre) motifs equally, whereas e2a protein required the promoter-proximal e(gre) motif. in t- and b-cell lines, the e(gre) motifs were of major importance for akv mlv transcriptional activity, while the ea/s motif had some effect. in contrast, neither e(gre) nor ea/s motifs contributed pronouncedly to akv mlv transcription in nih 3t3 cells lacking dna-binding alf1 or heb and e2a proteins. the id1 protein was found to repress alf1 activity in vitro and in vivo. moreover, ectopic id1 repressed e(gre)-directed but not ea/s-directed mlv transcription in lymphoid cell lines. in conclusion, e(gre) motifs and interacting basic helix-loop-helix proteins are important determinants for mlv transcriptional activity in lymphocytic cell lines."
1,PMID-1946356,"negative regulation of human immunodeficiency virus type 1 expression in monocytes: role of the 65-kda plus 50-kda nf-kappa b dimer. although monocytic cells can provide a reservoir for viral production in vivo, their regulation of human immunodeficiency virus type 1 (hiv-1) transcription can be either latent, restricted, or productive. these differences in gene expression have not been molecularly defined. in thp-1 cells with restricted hiv expression, there is an absence of dna-protein binding complex formation with the hiv-1 promoter-enhancer associated with markedly less viral rna production. this absence of binding was localized to the nf-kappa b region of the hiv-1 enhancer; the 65-kda plus 50-kda nf-kappa b heterodimer was preferentially lost. adding purified nf-kappa b protein to nuclear extracts from cells with restricted expression overcomes this lack of binding. in addition, treatment of these nuclear extracts with sodium deoxycholate restored their ability to form the heterodimer, suggesting the presence of an inhibitor of nf-kappa b activity. furthermore, treatment of nuclear extracts from these cells that had restricted expression with lipopolysaccharide increased viral production and nf-kappa b activity. antiserum specific for nf-kappa b binding proteins, but not c-rel-specific antiserum, disrupted heterodimer complex formation. thus, both nf-kappa b-binding complexes are needed for optimal viral transcription. binding of the 65-kda plus 50-kda heterodimer to the hiv-1 enhancer can be negatively regulated in monocytes, providing one mechanism restricting hiv-1 gene expression."
2,PMID-7543515,"costimulation of human cd4+ t cells with lfa-3 and b7 induce distinct effects on ap-1 and nf-kappa b transcription factors. we have earlier shown that stimulation of human cd4+ t cells with sea presented on chinese hamster ovary (cho)-dr transfectants coexpressing either b7 or lfa-3 resulted in distinct cytokine profiles. we now demonstrate that b7, but not lfa-3, strongly costimulated il-2 transcription and mrna expression in cd4+ t cells. maximal increase in il-2 transcription was recorded with cho-dr/b7/lfa-3, suggesting a cooperative effect of b7 and lfa-3 at the transcriptional level. gel-shift analysis demonstrated that stimulation of cd4+ t cells with cho-dr and staphylococcal enterotoxin a was sufficient to induce significant amounts of nf-ka

In [5]:
df_proteins = pd.DataFrame(total_proteins)
df_proteins.head()

,id,protein_type,protein_val
0,PMID-8943338,"protein *,* protein *,* protein *,* protein *,* protein *,* protein *,* protein *,* protein *,* protein","elf-1 *,* stat5 *,* interleukin-2 receptor alpha *,* interleukin 2 receptor alpha-chain *,* il-2r alpha *,* interleukin 2 *,* il-2 *,* hmg-i(y) *,* cat"
1,PMID-9224203,,
2,PMID-1653056,"protein *,* protein","tumor necrosis factor alpha *,* tnf alpha"
3,PMID-9872676,"protein *,* protein *,* protein *,* protein *,* protein *,* protein","interleukin-10 *,* inhibitory kappab-alpha *,* il-10 *,* tumor necrosis factor-alpha *,* tnf-alpha *,* ikappab-alpha"
4,PMID-8264604,"protein *,* protein *,* protein *,* protein","pu.1 *,* macrophage colony-stimulating factor receptor *,* macrophage colony-stimulating factor (m-csf) receptor *,* m-csf receptor"


In [6]:
df_entities = pd.DataFrame(total_entities)
df_entities.head()

,entity_type,entity_val,id
0,"entity *,* entity *,* entity *,* entity *,* entity *,* entity *,* entity *,* entity *,* entity","enhancer *,* promoter *,* promoters *,* enhancers *,* its own promoter. *,* immunoglobulin kappa enhancer *,* immunoglobulin heavy chain enhancer *,* kappa gene promoter *,* genes",PMID-1668145
1,"entity *,* entity *,* entity *,* entity","nf-kappa b *,* transcription factor *,* cytokine promoters *,* nf-kappa b-controlled cytokine promoters",PMID-9247567
2,"entity *,* entity *,* entity *,* entity *,* entity *,* entity","human immunodeficiency virus type 1 enhancer *,* site *,* kappa b sites *,* region *,* cd3 *,* cis-acting negative regulatory element",PMID-1533884
3,"entity *,* entity *,* entity","cytokine gene *,* cytokine genes *,* proinflammatory cytokine gene",PMID-8038234
4,"entity *,* entity *,* entity *,* entity","transcriptional complex *,* promoter *,* regulatory element *,* regulatory elements",PMID-7706710


## True Graph with Protein Entities Only

In [7]:
pro_entities = []
def reorganize_by_entity(row):
    val_list = list(filter(lambda x: x.strip(), row['protein_val'].split(' *,* ')))
    pro_entities.extend(list(map(lambda x:{'id':row['id'], 'val': x}, val_list)))

# %time df_proteins.apply(reorganize_by_entity, axis=1)

In [8]:
%time df_proteins.swifter.apply(reorganize_by_entity, axis=1)


CPU times: user 79.3 ms, sys: 5.27 ms, total: 84.5 ms
Wall time: 82.4 ms


0       None
1       None
2       None
3       None
4       None
5       None
6       None
7       None
8       None
9       None
        ... 
1050    None
1051    None
1052    None
1053    None
1054    None
1055    None
1056    None
1057    None
1058    None
1059    None
Length: 1060, dtype: object

In [9]:
df_pro_entities = pd.DataFrame(pro_entities)
df_pro_entities.head()

,id,val
0,PMID-8943338,elf-1
1,PMID-8943338,stat5
2,PMID-8943338,interleukin-2 receptor alpha
3,PMID-8943338,interleukin 2 receptor alpha-chain
4,PMID-8943338,il-2r alpha


In [10]:
df_pro_entities.loc[df_pro_entities.val=='']

,id,val


In [11]:
#ids = 800
len(df_pro_entities.id.unique())

1016

In [12]:
#values = 1540
len(df_pro_entities.val.unique())

1824

In [13]:
df_pro_entities.shape

(5239, 2)

In [14]:
df_sorted = df_pro_entities.groupby(df_pro_entities.val).size().reset_index(name='count_').sort_values(['count_'], ascending=False)
df_sorted.head()

,val,count_
850,il-2,158
1314,p50,109
1329,p65,106
1693,tnf-alpha,91
872,il-4,69


In [15]:
df_sorted.shape

(1824, 2)

In [16]:
once_appeared_values = df_sorted[df_sorted.count_==1].val.unique()
len(df_pro_entities.loc[df_pro_entities.val.isin(once_appeared_values)].id.unique())

572

In [17]:
copy_pro_entities = df_pro_entities.copy(deep=True)

In [18]:
copy_pro_entities.head()

,id,val
0,PMID-8943338,elf-1
1,PMID-8943338,stat5
2,PMID-8943338,interleukin-2 receptor alpha
3,PMID-8943338,interleukin 2 receptor alpha-chain
4,PMID-8943338,il-2r alpha


In [19]:
# def find_most_effective_recursively(copy_df, ordered_vals, i):
# #     print(ordered_vals[i])
#     found = copy_df.loc[copy_df.val==ordered_vals[i]]
# #     print(found)
#     if len(found.id.unique()) > 1:
#         return find_most_effective_recursively(copy_df.loc[copy_df.id.isin(found.id.unique())], ordered_vals, i+1)
#     elif len(found.id.unique()) ==1:
#         return found.iloc[0]
#     else:
# #         print(type(copy_df.groupby(copy_df.id).count().reset_index().sort_values(['val'], ascending=False).loc[0]))
#         return copy_df.groupby(copy_df.id).count().reset_index().sort_values(['val'], ascending=False).loc[0]
    
# # print(find_most_effective_recursively(copy_pro_entities, 3))

In [20]:
# def iterate_greedy(ordered_vals, _copy_pro_entities):

#     already_annotated = []
#     iter_no = 0
    
#     # 제일 빈도수 높은 entity를 많이 포함하고 있는 문서 부터 annotate
#     while(len(_copy_pro_entities)>0):
#         val = ordered_vals[0]

#         found_df = find_most_effective_recursively(_copy_pro_entities, ordered_vals, 0)
#         found_id = found_df.id
        
#         annotated_keywords = _copy_pro_entities.loc[_copy_pro_entities.id.eq(found_id)].val.unique()
#         if len(annotated_keywords)==0:
#             annotated_keywords.remove(val)
#             continue

#         already_annotated.append({'id': found_id, 
#                                   'keywords': annotated_keywords})

#         _copy_pro_entities = _copy_pro_entities.loc[(_copy_pro_entities.id!=found_id) & (~_copy_pro_entities.val.isin(annotated_keywords))]
#         _copy_pro_entities.sort_index(inplace=True)

#         for an in annotated_keywords:
#             ordered_vals.remove(an)
#     #     print(len(ordered_vals))
#         iter_no += 1

#     print(iter_no)
#     return iter_no
iterate_greedy(list(df_sorted.val.unique()), copy_pro_entities)

676

## PageRank

In [21]:
# def generate_edge_weights(_df_pro_entities):
#     p_edges = {}
#     for val, grp in _df_pro_entities.groupby('val'):
#         if len(grp) > 1:
#             id_list = grp.id.unique()
#             for i in range(len(id_list)-1):
#                 if id_list[i]+'|'+id_list[i+1] not in p_edges.keys():
#                     p_edges[id_list[i]+'|'+id_list[i+1]] = 1
#                 else:
#                     p_edges[id_list[i]+'|'+id_list[i+1]] += 1

#                 if id_list[i+1]+'|'+id_list[i] not in p_edges.keys():
#                     p_edges[id_list[i+1]+'|'+id_list[i]] = 1
#                 else:
#                     p_edges[id_list[i+1]+'|'+id_list[i]] += 1
#     return p_edges

# def build_graph(_df_pro_entities):
#     G = nx.DiGraph()

#     for k in _df_pro_entities.id.unique():
#         G.add_node(k)
    
#     p_edges = generate_edge_weights(_df_pro_entities)
#     for key, val in p_edges.items():
#     #     print(key)
#         keys = key.split('|')
#         G.add_edge(keys[0], keys[1], weight=val)

#     return G
    
G = build_graph(df_pro_entities)

In [22]:
# for u, v, d in G.edges(data=True):
#     print(d)
len(G.edges)

5176

In [23]:
G.has_edge('PMID-8943338', 'PMID-7802642')
# G['PMID-7721885']['PMID-7802642']
G['PMID-8283032']

True

AtlasView({'PMID-8455941': {'weight': 1}, 'PMID-8955000': {'weight': 1}, 'PMID-7512079': {'weight': 1}, 'PMID-9024987': {'weight': 3}, 'PMID-9686612': {'weight': 1}, 'PMID-7635572': {'weight': 1}, 'PMID-10225979': {'weight': 1}, 'PMID-8206753': {'weight': 4}, 'PMID-7489741': {'weight': 1}, 'PMID-7481768': {'weight': 1}, 'PMID-8709636': {'weight': 2}, 'PMID-10231345': {'weight': 1}, 'PMID-9467376': {'weight': 1}, 'PMID-10224223': {'weight': 1}, 'PMID-8871061': {'weight': 1}})

In [24]:
G.get_edge_data('PMID-7721885', 'PMID-9388475')['weight']
G.get_edge_data('PMID-7721885', 'PMID---') is None

1

True

In [25]:
# nx.draw_networkx(G, pos=nx.circular_layout(G), with_labels=True)

In [26]:
# pageranked_list = nx.pagerank(G, alpha=0.0001)
pageranked_list = compute_pagerank(df_pro_entities)
max(pageranked_list.items(), key=operator.itemgetter(1))

('PMID-7721885', 0.0009845714930966205)

In [27]:
min(pageranked_list.items(), key=operator.itemgetter(1))

('PMID-8195215', 0.0009841598401946804)

In [28]:
# sorted_pagedranked = sorted(pageranked_list, key=operator.itemgetter(1))
sorted_pagedranked = sorted(pageranked_list.items(), key=operator.itemgetter(1), reverse=True)
sorted_pagedranked[:10]

[('PMID-7721885', 0.0009845714930966205),
 ('PMID-10339475', 0.0009845538925006422),
 ('PMID-10233888', 0.0009845537398208634),
 ('PMID-10097788', 0.0009845115132501513),
 ('PMID-8513868', 0.000984501088786421),
 ('PMID-8641346', 0.0009844867915132654),
 ('PMID-8871617', 0.0009844771382728054),
 ('PMID-8083467', 0.0009844733209045816),
 ('PMID-1423591', 0.000984464444547021),
 ('PMID-8978306', 0.0009844630906406474)]

In [29]:
pageranked_list['PMID-8283032']

0.0009843788694108581

In [30]:
len(sorted_pagedranked)

1016

In [31]:
sort_ranked = list(map(lambda x: x[0], sorted_pagedranked))
sort_ranked[:5]

['PMID-7721885',
 'PMID-10339475',
 'PMID-10233888',
 'PMID-10097788',
 'PMID-8513868']

## PageRanked order -> Count the number of iteration

In [32]:
# def iterate_pagerank(_ordered_vals, _df_pro_entities):
#     pageranked_list = compute_pagerank(_df_pro_entities)

#     sort_ranked = list(map(lambda x: x[0], sorted(pageranked_list.items(), key=operator.itemgetter(1), reverse=True)))
    
#     copy_pro_entities = _df_pro_entities.copy(deep=True)

#     already_annotated = []
#     iter_no = 0
#     i = 0

#     while(len(copy_pro_entities)>0):
#         found_id = sort_ranked[0]

#         if len(copy_pro_entities.loc[copy_pro_entities.id.eq(found_id)])==0:
#             sort_ranked.remove(found_id)
#             continue
#         annotated_keywords = copy_pro_entities.loc[copy_pro_entities.id.eq(found_id)].val.unique()        
#         already_annotated.append({'id': found_id, 
#                                   'keywords': annotated_keywords})

#         copy_pro_entities = copy_pro_entities.loc[(copy_pro_entities.id!=found_id) & (~copy_pro_entities.val.isin(annotated_keywords))]
#         copy_pro_entities.sort_index(inplace=True)
#         sort_ranked.remove(found_id)

#         for an in annotated_keywords:
#             _ordered_vals.remove(an)

#         iter_no += 1
#     print(iter_no)
#     return iter_no

iterate_pagerank(list(df_sorted.val.unique()), df_pro_entities)

670

In [33]:
# 'PMID-8943338' in copy_pro_entities.id.unique()
# list(filter(lambda x: x[0] in copy_pro_entities.id.unique(), [('PMID-8513868', 0.000984501088786421), ('PMID-8641346', 0.0009844867915132654), ('PMID-8871617', 0.0009844771382728054), ('PMID-8083467', 0.0009844733209045816), ('PMID-1423591', 0.000984464444547021)]))

In [34]:
len(copy_pro_entities)

5239

### Random Order

In [35]:
copy_pro_entities = df_pro_entities.copy(deep=True)
copy_pro_entities.shape

(5239, 2)

In [36]:
len(copy_pro_entities.id.unique())

1016

In [37]:
len(df_sent.id.unique())

1060

In [38]:
# def iterate_random(_ordered_vals, _df_pro_entities, _id_list):
#         copy_pro_entities = _df_pro_entities.copy(deep=True)

#         already_annotated = []
#         iter_no = 0
#         while(len(copy_pro_entities)>0):
#             ran_id = choice(_id_list)
#             _id_list.remove(ran_id)

#             if len(copy_pro_entities.loc[copy_pro_entities.id.eq(ran_id)])==0:
#                 continue

#             annotated_keywords = copy_pro_entities.loc[copy_pro_entities.id.eq(ran_id)].val.unique()
#             already_annotated.append({'id': ran_id, 
#                                       'keywords': annotated_keywords})

#             copy_pro_entities = copy_pro_entities.loc[(copy_pro_entities.id!=ran_id) & (~copy_pro_entities.val.isin(annotated_keywords))]
#             copy_pro_entities.sort_index(inplace=True)

#             for an in annotated_keywords:
#                 _ordered_vals.remove(an)
#             iter_no += 1
        
#         print(iter_no)
#         return iter_no
    
for i in range(10):
    iterate_random(list(df_sorted.val.unique()), df_pro_entities, list(df_sent.id.unique()))

713

692

688

701

707

685

695

698

693

692

## DDI Dataset

In [61]:
file_locs = ['/Users/grace/workspace/bio_dataset/semeval_task9_train_pair/Train/MedLine',
            '/Users/grace/workspace/bio_dataset/semeval_task9_train_pair/Train/DrugBank']

entities_ddi = []
docs = []
# entity_types = set()
for file_loc in file_locs:
    medlines = os.listdir(file_loc)

    for med in medlines:
        sents = []
        entities = []
        ent_types = []
        
        doc = minidom.parse(file_loc+'/'+med)
        items = doc.getElementsByTagName('document')
        doc_id = items[0].attributes['id'].value

        for sent in doc.getElementsByTagName('sentence'):
            sent_id = sent.attributes['id'].value
            sent_text = sent.attributes['text'].value
            if sent_text.strip():
                sents.append(sent_text)

            # entity(drug)의 갯수가 1보다 클때만
            if len(sent.getElementsByTagName('entity')) > 0:
                for en in sent.getElementsByTagName('entity'):
                    e_type = en.attributes['type'].value
#                     entity_types.add(e_type)
                    ent_types.append(e_type)
                    e_text = en.attributes['text'].value
                    entities.append(e_text)
#                     'entity_val': ' *,* '.join(entities.keys())
                    
        docs.append({'id':doc_id, 'sent':'\n'.join(sents)})
        entities_ddi.append({'id':doc_id, 'drug_type':' *,* '.join(ent_types), 'drug_val':' *,* '.join(entities)})

In [65]:
df_sent_ddi = pd.DataFrame(docs)
df_sent_ddi.head()
df_sent_ddi.shape

,id,sent
0,DDI-MedLine.d69,"Differential regulation of tyrosine phosphorylation in tumor cells by contortrostatin, a homodimeric disintegrin, and monomeric disintegrins echistatin and flavoridin.\nThe homodimeric disintegrin contortrostatin was compared directly to the monomeric disintegrins echistatin and flavoridin for the ability to affect protein tyrosine phosphorylation in tumor cells. \nIt was observed that contortrostatin had a dramatic effect on the tyrosine phosphorylation status of several proteins in T24 human bladder cancer cells, including robust induction of phosphorylation of proteins in the range of 120-140 kDa. \nEchistatin alone had no effect on tyrosine phosphorylation in T24 cells, but dose-dependently inhibits the effects of contortrostatin when both are added simultaneously. \nAmong the proteins that undergo tyrosine phosphorylation in response to contortrostatin treatment is CAS, a 130 kDa adapter protein involved in integrin signaling. \nFlavoridin alone was found to have no effect on CAS, but can completely block contortrostatin-induced phosphorylation of this protein in MDA-MB-435 cells. \nThese observations strongly suggest that the homodimeric structure of contortrostatin functionally distinguishes it from other monomeric members of the disintegrin family."
1,DDI-MedLine.d141,"Toxicity of cadmium and zinc to encystment and in vitro excystment of Parorchis acanthus (Digenea: Philophthalmidae).\n\nThe toxicity of cadmium, zinc and cadmium/zinc mixtures at concentrations ranging from 1000 to 50000 microg/l were investigated against cercariae and metacercariae of Parorchis acanthus obtained from the dog whelk Nucella lapillus. \nCercarial encystment at concentrations of 25000 microg/l or higher was significantly impaired by all test metals; \nhowever, at lower concentrations only zinc demonstrated toxicity. \nMixtures of cadmium and zinc had a synergistic effect compared with single metal toxicity but only at 50000 microg/l. \nExcystment in vitro was only significantly affected by cercariae exposed to cadmium/zinc mixtures whilst encysting. \nTwenty-four h exposures of fully formed cysts had no effect on excystment in vitro. \nEffects on in vitro excystment rates over a 2 h period demonstrated widespread effects for cercariae-exposed P. \nacanthus. \nNo effects were evident on excystment rates of cyst-exposed parasites."
2,DDI-MedLine.d125,"The mode of toxic action of the pesticide gliftor: the metabolism of 1,3-difluoroacetone to (-)-erythro-fluorocitrate.\r\n\nThe biochemical toxicology of 1,3-difluoroacetone, a known metabolite of the major ingredient of the pesticide Gliftor (1,3-difluoro-2-propanol), was investigated in vivo and in vitro. \nRat kidney homogenates supplemented with coenzyme A, ATP, oxaloacetate, and Mg2+ converted 1,3-difluoroacetone to (-)-erythro-fluorocitrate in vitro. \nAdministration of 1,3-difluoroacetone (100 mg kg(-1) body weight) to rats in vivo resulted in (-)-erythro-fluorocitrate synthesis in the kidney, which was preceded by an elevation in fluoride levels and followed by citrate accumulation. \nAnimals dosed with 1,3-difluoroacetone did not display the 2-3 hour lag phase in either (-)-erythro-fluorocitrate synthesis or in citrate and fluoride accumulation characteristic of animals dosed with 1,3-difluoro-2-propanol. \nWe demonstrate that the conversion of 1,3-difluoro-2-propanol to 1,3-difluoroacetone by an NAD+-dependent oxidation is the rate-limiting step in the synthesis of the toxic product, (-)-erythro-fluorocitrate from 1,3-difluoro-2-propanol. \nPrior administration of 4-methylpyrazole (90 mg kg(-1) body weight) was shown to prevent the conversion of 1,3-difluoro-2-propanol (100 mg kg(-1) body weight) to (-)-erythro-fluorocitrate in vivo and to eliminate the fluoride and citrate elevations seen in 1,3-difluoro-2-propanol-intoxicated animals. \nHowever, administration of 4-methylpyrazole (90 mg kg(-1) body weight) to rats 2 hours prior to 1,3-difluoroacetone (100

(714, 2)

In [66]:
df_drugs = pd.DataFrame(entities_ddi)
df_drugs.head()

,drug_type,drug_val,id
0,"drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n","contortrostatin *,* echistatin *,* flavoridin *,* contortrostatin *,* echistatin *,* flavoridin *,* contortrostatin *,* Echistatin *,* contortrostatin *,* contortrostatin *,* Flavoridin *,* contortrostatin *,* contortrostatin",DDI-MedLine.d69
1,"drug *,* drug *,* drug *,* drug","zinc *,* zinc *,* zinc *,* zinc",DDI-MedLine.d141
2,"drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug_n *,* drug *,* drug_n *,* drug_n *,* drug_n *,* drug *,* drug_n *,* drug_n *,* drug *,* drug_n *,* drug_n *,* drug_n","gliftor *,* 1,3-difluoroacetone *,* (-)-erythro-fluorocitrate *,* 1,3-difluoroacetone *,* Gliftor *,* 1,3-difluoro-2-propanol *,* 1,3-difluoroacetone *,* (-)-erythro-fluorocitrate *,* 1,3-difluoroacetone *,* (-)-erythro-fluorocitrate *,* 1,3-difluoroacetone *,* (-)-erythro-fluorocitrate *,* 1,3-difluoro-2-propanol *,* 1,3-difluoro-2-propanol *,* 1,3-difluoroacetone *,* (-)-erythro-fluorocitrate *,* 1,3-difluoro-2-propanol *,* 4-methylpyrazole *,* 1,3-difluoro-2-propanol *,* (-)-erythro-fluorocitrate *,* 1,3-difluoro-2-propanol *,* 4-methylpyrazole *,* 1,3-difluoroacetone *,* (-)-erythro-fluorocitrate *,* 4-methylpyrazole *,* 1,3-difluoro-2-propanol *,* 1,3-difluoro-2-propanol *,* 1,3-difluoroacetone",DDI-MedLine.d125
3,"drug *,* drug_n *,* drug *,* group *,* drug *,* drug_n *,* drug *,* drug_n *,* drug *,* drug *,* drug_n *,* drug *,* drug_n *,* drug *,* drug *,* drug *,* drug_n *,* drug *,* drug *,* drug_n *,* drug *,* drug_n *,* drug *,* drug_n *,* drug","naloxone *,* beta-endorphin *,* morphine *,* opioids *,* naloxone *,* beta-endorphin *,* morphine *,* beta-endorphin *,* morphine *,* naloxone *,* beta-endorphin *,* naloxone *,* beta-endorphin *,* naloxone *,* morphine *,* naloxone *,* beta-endorphin *,* morphine *,* naloxone *,* beta-endorphin *,* morphine *,* beta-endorphin *,* morphine *,* beta-endorphin *,* morphine",DDI-MedLine.d63
4,group,topoisomerase-targeting antitumor drugs,DDI-MedLine.d33


In [69]:
drug_entities = []
def reorganize_by_entity(row):
    val_list = list(filter(lambda x: x.strip(), row['drug_val'].split(' *,* ')))
    drug_entities.extend(list(map(lambda x:{'id':row['id'], 'val': x}, val_list)))

%time df_drugs.swifter.apply(reorganize_by_entity, axis=1)


CPU times: user 149 ms, sys: 5.3 ms, total: 155 ms
Wall time: 152 ms


0      None
1      None
2      None
3      None
4      None
5      None
6      None
7      None
8      None
9      None
       ... 
704    None
705    None
706    None
707    None
708    None
709    None
710    None
711    None
712    None
713    None
Length: 714, dtype: object

In [70]:
df_drug_entities = pd.DataFrame(drug_entities)
df_drug_entities.head()

,id,val
0,DDI-MedLine.d69,contortrostatin
1,DDI-MedLine.d69,echistatin
2,DDI-MedLine.d69,flavoridin
3,DDI-MedLine.d69,contortrostatin
4,DDI-MedLine.d69,echistatin


In [72]:
df_sorted = df_drug_entities.groupby(df_drug_entities.val).size().reset_index(name='count_').sort_values(['count_'], ascending=False)
df_sorted.head()

,val,count_
2956,warfarin,212
1906,digoxin,209
2572,phenytoin,176
2838,theophylline,146
2278,lithium,120


In [76]:
for i in range(10):
    copy_drug_entities = df_drug_entities.copy(deep=True)
    
    print('greedy')
    iterate_greedy(list(df_sorted.val.unique()), copy_drug_entities)
    print('pagerank')    
    iterate_pagerank(list(df_sorted.val.unique()), copy_drug_entities)
    print('random')
    iterate_random(list(df_sorted.val.unique()), copy_drug_entities, list(df_sent_ddi.id.unique()))

greedy


605

pagerank


596

random


...

605

pagerank


596

random


621

In [110]:
file_locs = ['/Users/grace/workspace/bio_dataset/CONLL2003']


entity_types = set()
entities_cornell = []
sent_id = 0
for file_loc in file_locs:
    medlines = list(filter(lambda x: x.endswith('.txt'), os.listdir(file_loc)))

    sents = []
    for med in medlines:
        f = open(file_loc+'/'+med, 'r')
        lines = f.readlines()[1:]
        
        sent = ''
        entities = []
        ent_types = []
        for l in lines:
            l = l.replace('\n', '')
            if l.strip():
                ll = l.split(' ')
                sent += ' '+ll[0]
                entity_types.add(ll[3])
                
                if ll[3].startswith('B'):
                    entities.append(ll[0])
                    ent_types.append(ll[3].replace('B-', ''))
                elif ll[3].startswith('I'):
                    entities[len(entities)-1] = entities[len(entities)-1] + ' ' + ll[0]

                if ll[0]=='.':
                    sents.append({'id':'cor_'+str(sent_id), 'sent':sent.strip()})
                    sent = ''
                    sent_id += 1
                    entities_cornell.append({'id':'cor_'+str(sent_id), 'en_type':' *,* '.join(ent_types), 'en_val':' *,* '.join(entities)})
                    
                    entities = []
                    ent_types = []

In [111]:
entity_types

{'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}

In [112]:
df_sent_cornell = pd.DataFrame(sents)
df_sent_cornell.head()
df_sent_cornell.shape

,id,sent
0,cor_0,EU rejects German call to boycott British lamb .
1,cor_1,Peter Blackburn BRUSSELS 1996-08-22 The European Commission said on Thursday it disagreed with German advice to consumers to shun British lamb until scientists determine whether mad cow disease can be transmitted to sheep .
2,cor_2,Germany 's representative to the European Union 's veterinary committee Werner Zwingmann said on Wednesday consumers should buy sheepmeat from countries other than Britain until the scientific advice was clearer .
3,cor_3,""" We do n't support any such recommendation because we do n't see any grounds for it , "" the Commission 's chief spokesman Nikolaus van der Pas told a news briefing ."
4,cor_4,He said further scientific study was required and if it was found that action was needed it should be taken by the European Union .


(10874, 2)

In [120]:
len(df_sent_cornell.id.unique())

10874

In [113]:
df_cornell = pd.DataFrame(entities_cornell)
df_cornell.head()

,en_type,en_val,id
0,"ORG *,* MISC *,* MISC","EU *,* German *,* British",cor_1
1,"PER *,* LOC *,* ORG *,* MISC *,* MISC","Peter Blackburn *,* BRUSSELS *,* European Commission *,* German *,* British",cor_2
2,"LOC *,* ORG *,* PER *,* LOC","Germany *,* European Union *,* Werner Zwingmann *,* Britain",cor_3
3,"ORG *,* PER","Commission *,* Nikolaus van der Pas",cor_4
4,ORG,European Union,cor_5


In [114]:
cornell_entities = []
def reorganize_by_entity(row):
    val_list = list(filter(lambda x: x.strip(), row['en_val'].split(' *,* ')))
    cornell_entities.extend(list(map(lambda x:{'id':row['id'], 'val': x}, val_list)))

%time df_cornell.swifter.apply(reorganize_by_entity, axis=1)


CPU times: user 506 ms, sys: 7.56 ms, total: 513 ms
Wall time: 510 ms


0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
         ... 
10864    None
10865    None
10866    None
10867    None
10868    None
10869    None
10870    None
10871    None
10872    None
10873    None
Length: 10874, dtype: object

In [115]:
df_cor_entities = pd.DataFrame(cornell_entities)
df_cor_entities.head()

,id,val
0,cor_1,EU
1,cor_1,German
2,cor_1,British
3,cor_2,Peter Blackburn
4,cor_2,BRUSSELS


In [116]:
df_sorted = df_cor_entities.groupby(df_cor_entities.val).size().reset_index(name='count_').sort_values(['count_'], ascending=False)
df_sorted.head()

,val,count_
10252,U.S.,550
3750,Germany,291
887,Australia,285
3471,France,247
9447,Spain,223


In [130]:
'cor_10874' in list(df_sent_cornell.id.unique())

False

In [133]:
id_list = list(df_sent_cornell.id.unique())
for i in range(1):
    copy_cor_entities = df_cor_entities.copy(deep=True)
    
    print('greedy')
    iterate_greedy(list(df_sorted.val.unique()), copy_cor_entities)
    print('pagerank')    
    iterate_pagerank(list(df_sorted.val.unique()), copy_cor_entities)
    print('random')
    iterate_random(list(df_sorted.val.unique()), df_cor_entities, id_list)

greedy


4738

pagerank


4500

random


4860

In [ ]:
# Sang, Erik F., and Fien De Meulder. "Introduction to the CoNLL-2003 shared task: Language-independent named entity recognition." arXiv preprint cs/0306050 (2003).